# Entrenamiento de detección de objetos con dataset propio
En este notebook entrenaremos una arquitectura Yolo-v4 para detectar objetos de un dataset propio. Usaremos transfer learning y fine-tuning para hacer que un modelo pre-entrenado con COCO dataset aprenda los nuevos objetos del dataset propio.

Por favor seguir las instrucciones a continuación

# Configuración de Yolo-v4
Clonamos el repositorio con el código fuente.

In [ ]:
!git clone https://github.com/ivansipiran/pytorch-YOLOv4.git

Instalamos los requerimientos necesarios para que funcione el código fuente.

In [ ]:
%cd /content/pytorch-YOLOv4
!pip install -r requirements.txt

Descargamos los pesos del detector pre-entrenado. El modelo ya se encuentra almacenado en formato de Pytorch.

In [ ]:
%cd /content/pytorch-YOLOv4
!gdown https://drive.google.com/uc?id=1fcbR0bWzYfIEdLJPzOsn4R5mlvR6IQyA

#Dataset

Descargar el dataset para el entrenamiento. Este dataset contiene piezas de ajedrez que queremos aprender a detectar. El modelo pre-entrenado no tuvo a este tipo de objetos en los datos del pre-entrenamiento, así que la idea es aprender estos nuevos objetos.

In [ ]:
!wget http://www.ivan-sipiran.com/downloads/datasetchess.zip
!unzip datasetchess.zip

Miremos algunos ejemplos de los datos. Hay tres folders (train, test, valid) con imágenes. Cada folder contiene un archivo con las anotaciones de los objetos a detectar. En este código, mostramos una imagen y sus anotaciones.

In [ ]:
#Chequearemos algunos datos del ground-truth
import random
import seaborn as sns
import cv2
import matplotlib.pyplot as plt

folder = 'train'

#Leemos el archivo de anotaciones
lineList = []
with open(folder + '/_annotations.txt') as f:
    line = f.readline()
    lineList.append(line.strip())
    while line:
        line = f.readline()
        if len(line)> 0:
            lineList.append(line.strip())

#Leer las clases dentro del folder
classList = []
with open(folder + '/_classes.txt') as f:
    line = f.readline()
    classList.append(line.strip())
    while line:
        line = f.readline()
        if len(line)>0:
            classList.append(line.strip())

numClasses = len(classList)
print(f'Número de clases:{numClasses}')

randomTest = random.choice(lineList)
annotations = randomTest.split()

palette = sns.color_palette(None, numClasses)

img = cv2.imread(folder+'/'+annotations[0])

for i in range(1,len(annotations)):
    bboxData = annotations[i].split(',')
    color = palette[int(bboxData[4])]
    print(color)
    cv2.rectangle(img, (int(bboxData[0]), int(bboxData[1])),(int(bboxData[2]), int(bboxData[3])), (int(color[0]*255),int(color[1]*255), int(color[2]*255)), 2, 1)

plt.figure(figsize=(10,10))
plt.imshow(img[:,:,::-1])
plt.show()


In [ ]:
#Copiar los archivos de anotaciones a los folders de Yolo

%cp train/_annotations.txt train/train.txt
%cp train/_annotations.txt train.txt
%cp valid/_annotations.txt data/val.txt
%cp valid/*.jpg train/

In [ ]:
# El número de clases es el número de líneas del archivo "_classes.txt"
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/_classes.txt')

In [8]:
print(num_classes)

2


# Entrenar el detector propio

In [ ]:
#Entrenamiento
#-b batch size (mantener este valor bajo (2-4))
#-s número de subdivisiones en el batch, relevante en el framework darknet
#-l learning rate
#-g GPU device
#pretrained indicar conde se encuentran los pesos pre-entrenados
#classes - número de clases
#dir - dónde está la data de entrenamiento
#epoch - número de épocas
!python train.py -b 2 -s 1 -l 0.001 -g 0 -pretrained ./yolov4.conv.137.pth -classes {num_classes} -dir ./train -epochs 50

# Cargar pesos entrenados y probar el detector

Cargamos los pesos!

In [ ]:
#En esta carpeta deben estar todos los checkpoints guardados durante el entrenamiento
!ls checkpoints

In [ ]:
#Escoger una imagen aleatoria
import os
test_images = [f for f in os.listdir('test') if f.endswith('.jpg')]
import random
img_path = "test/" + random.choice(test_images);

In [ ]:
%%time
##Usar el checkpoint que deseas para probar la inferencia
!python models.py {num_classes} checkpoints/Yolov4_epoch46.pth {img_path} test/_classes.txt

In [ ]:
#Después de la inferencia, el resultado se encuentra en la imagen "predictions.jpg"
#Visualizar la inferencia
from IPython.display import Image
Image('predictions.jpg')

# Ejercicio

Usar el dataset de detección de mascarillas: [www.ivan-sipiran.com/downloads/maskwearing6.zip](www.ivan-sipiran.com/downloads/maskwearing6.zip). Intenta diferentes parámetros, este dataset es más difícil.

También podrías anotar tu propio dataset usando herramientas como: [LabelImg](https://github.com/tzutalin/labelImg), o [VoTT](https://github.com/microsoft/VoTT). Probablmente necesites cambiar el formato de los datos dependiendo del modelo que quieras entrenar.